In [1]:
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
matplotlib.style.use('ggplot')
from patsy import dmatrices
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
file_name='marketing_training.csv'

In [3]:
df = pd.read_csv(file_name)

In [4]:
df.rename(columns={'emp.var.rate':'emp_var_rate'}, inplace=True)
df.rename(columns={'cons.price.idx':'cons_price_idx'}, inplace=True)
df.rename(columns={'cons.conf.idx':'cons_conf_idx'}, inplace=True)
df.rename(columns={'nr.employed':'nr_employed'},inplace=True)

In [5]:
df.columns

Index([u'custAge', u'profession', u'marital', u'schooling', u'default',
       u'housing', u'loan', u'contact', u'month', u'day_of_week', u'campaign',
       u'pdays', u'previous', u'poutcome', u'emp_var_rate', u'cons_price_idx',
       u'cons_conf_idx', u'euribor3m', u'nr_employed', u'pmonths',
       u'pastEmail', u'responded'],
      dtype='object')

In [6]:
#Filling Missing value Function based on correlation of other columns. 
def fill_null_value(df_temp,based_col,target_col,length=1,numerical=True):
    if length==2:
        a1=df_temp[based_col[0]].unique()
        for val in a1:
            df_first=df_temp[df_temp[based_col[0]]==val]
            count_val = df_first[based_col[1]].value_counts()
            second_list =dict(count_val[count_val>50]).keys()
            if second_list!=[]:
                for val1 in second_list:
                    df_second = df_first[df_first[based_col[1]]==val1]
                    if numerical:
                        mean_val=df_second[df_second[target_col].notnull()][target_col].mean()
                        index_fill = df_second[df_second[target_col].isnull()][target_col].index
                        df_temp.ix[index_fill,target_col]=mean_val
                    if not numerical: 
                        df_s=df_second[df_second[target_col].notnull()][target_col]
                        high_val = df_s.value_counts().idxmax()
                        index_fill = df_second[df_second[target_col].isnull()][target_col].index
                        df_temp.ix[index_fill,target_col]=high_val
        
        
        
    if length==1:
        a1 = df_temp[based_col[0]].unique()
        for val in a1:
            df_first=df_temp[df_temp[based_col[0]]==val]
            if numerical:
                mean_val=df_first[df_first[target_col].notnull()][target_col].mean()
                index_fill = df_first[df_first[target_col].isnull()][target_col].index
                df_temp.ix[index_fill,target_col]=mean_val
            if not numerical: 
                df_s=df_first[df_first[target_col].notnull()][target_col]
                high_val = df_s.value_counts().idxmax()
                index_fill = df_first[df_first[target_col].isnull()][target_col].index
                df_temp.ix[index_fill,target_col]=high_val
            
   
                    
                        
    return df_temp
        
        
        

In [7]:
#Filling Null values for customer Age, based on the marital status and Profession
df1=df.copy(deep=True)
df=fill_null_value(df1,['marital','profession'],'custAge',length=2)
df['custAge']=df['custAge'].fillna(df['custAge'].mean())

In [8]:
#Filling missing values for schooling based on profession
df1 = df.copy(deep=True)
df = fill_null_value(df1,['profession'],'schooling',length=1,numerical=False)

In [9]:
#filling missing values for day_of_week based on profession they are in
df1 = df.copy(deep=True)
df = fill_null_value(df1,['profession'],'day_of_week',length=1,numerical=False)


In [10]:
df.responded.value_counts()


no     6574
yes     840
Name: responded, dtype: int64

In [11]:
df.shape

(7414, 22)

In [12]:
df['responded'].value_counts(dropna=True)

no     6574
yes     840
Name: responded, dtype: int64

In [13]:
#Categorical Values to be bucketed : campaign, pdays, month, profession, previous, emp.var.rate,pastEmail
#Categorical Values : ['profession','marital','schooling','default','housing','contact','']
#Floating values : cusAge, cons.conf.idx, euribor3m, nr.employed

In [14]:
# Bucketizing the data based on the distribution 

#######################
#campaign
df['campaign'].value_counts()

d_cam = 'campaign_derived'
df[d_cam]   = df['campaign']
##########
c0f = (df[d_cam]<=1)
c1f = (df[d_cam]>1) & (df[d_cam]<=4)
c2f = (df[d_cam]>4) & (df[d_cam]<=7)
c3f = (df[d_cam]>7) & (df[d_cam]<=14)
c4f = (df[d_cam]>14)
##########

df.ix[c0f,d_cam] = '<2'
df.ix[c1f,d_cam] = '2-4'
df.ix[c2f,d_cam] = '4-7'
df.ix[c3f,d_cam] = '7-14'
df.ix[c4f,d_cam] = '>14'


In [15]:
#######################
#pdays


pdays_d = 'pdays_derived'
df[pdays_d]   = df['pdays']
##########
c0f = (df[pdays_d]<=2)
c1f = (df[pdays_d]>2) & (df[pdays_d]<=4)
c2f = (df[pdays_d]>4) & (df[pdays_d]<=7)
c3f = (df[pdays_d]>7)
##########

df.ix[c0f,pdays_d] = '<3'
df.ix[c1f,pdays_d] = '3-4'
df.ix[c2f,pdays_d] = '5-7'
df.ix[c3f,pdays_d] = '>7'


In [16]:
#######################
#month

month_d = 'month_derived'
df[month_d]   = df['month']
##########
c0f = (df[month_d].isin(['jan','feb','mar']))
c1f = (df[month_d].isin(['apr','may','jun']))
c2f = (df[month_d].isin(['jul','aug','sep']))
c3f = (df[month_d].isin(['oct','nov','dec']))
##########

df.ix[c0f,month_d] = 'q1'
df.ix[c1f,month_d] = 'q2'
df.ix[c2f,month_d] = 'q3'
df.ix[c3f,month_d] = 'q4'



In [17]:
#######################
#previous

month_d = 'month_derived'
df[month_d]   = df['month']
##########
c0f = (df[month_d].isin(['jan','feb','mar']))
c1f = (df[month_d].isin(['apr','may','jun']))
c2f = (df[month_d].isin(['jul','aug','sep']))
c3f = (df[month_d].isin(['oct','nov','dec']))
##########

df.ix[c0f,month_d] = 'q1'
df.ix[c1f,month_d] = 'q2'
df.ix[c2f,month_d] = 'q3'
df.ix[c3f,month_d] = 'q4'


In [18]:
#########
df['previous'].value_counts()
previous_d = 'previous_derived'
df[previous_d]   = df['previous']
##########
c0f = (df[previous_d]<=0)
c1f = (df[previous_d]>0) & (df[previous_d]<=2)
c2f = (df[previous_d]>2)
##########

df.ix[c0f,previous_d] = '<1'
df.ix[c1f,previous_d] = '1-2'
df.ix[c2f,previous_d] = '>2'




In [19]:
#########
df['previous'].value_counts()
pmail_d = 'pastEmail_derived'
df[pmail_d]   = df['pastEmail']
##########
c0f = (df[pmail_d]<=0)
c1f = (df[pmail_d]>0) & (df[pmail_d]<=4)
c2f = (df[pmail_d]>4)
##########

df.ix[c0f,pmail_d] = '<1'
df.ix[c1f,pmail_d] = '1-4'
df.ix[c2f,pmail_d] = '>4'


In [20]:
####################responded to numerical 
responded__d = 'responded_derived'
df[responded__d]   = df['responded']
##########
c1f = df[responded__d]=='yes'
c0f = df[responded__d]=='no'


df.ix[c0f,responded__d] = 0
df.ix[c1f,responded__d] = 1


In [50]:
#Selecting columns for the model building 
colums_model = ['custAge','emp_var_rate',"cons_price_idx",'cons_conf_idx','nr_employed','euribor3m','profession','marital','schooling','housing','loan','contact','month_derived','day_of_week','campaign_derived',
                'pdays_derived','previous_derived','poutcome','pastEmail_derived','responded_derived','responded']
df_model = df[colums_model]

In [49]:
#df_model.to_csv('final_modified_data.csv')


In [23]:
y,X=dmatrices('responded ~ custAge+emp_var_rate+cons_price_idx+cons_conf_idx+nr_employed+euribor3m+profession+marital+schooling+housing+loan+contact+month_derived+day_of_week+campaign_derived+pdays_derived+previous_derived+poutcome+pastEmail_derived',df_model,return_type="dataframe")

In [24]:
y=df_model['responded_derived']
y=np.array(map(int,y))

In [ ]:
#Splitting the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [27]:
#Logistic Regression using class balance technique 
model = LogisticRegression(class_weight='auto')
model.fit(X_train,y_train)
X_test_predict=model.predict(X_test)
print metrics.confusion_matrix(y_test,X_test_predict)

/usr/local/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


LogisticRegression(C=1.0, class_weight='auto', dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#This is the code for Random Forest Classifier. 





In [31]:
# Using Random Forest Trees
from sklearn.ensemble import RandomForestClassifier
model_r = RandomForestClassifier(class_weight='auto',n_estimators=100)
model_r.fit(X_train,y_train)
y_test_predict_random=model_r.predict(X_test)
print metrics.confusion_matrix(y_test,y_test_predict_random)

RandomForestClassifier(bootstrap=True, class_weight='auto', criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [ ]:
# USing Support Vector Machines 
from sklearn.svm import SVC
model_svc = SVC(kernel='rbf',C=10,class_weight='auto')
model_svc.fit(X_train,y_train)
y_test_predict_svc=model_svc.predict(X_test)
metrics.confusion_matrix(y_test,y_test_predict_svc)